# Project 3: Blood Donor Classification
(by: Martin Marsal, Benedikt Allmendinger, Christian Diegmann; Heilbronn University, Germany, January 2025)

This project implements a machine learning pipeline that helps hospital staff to decide whether a
person can be a blood donor.

The following references were used:

Some of the lectures from "Machine Learning (172336)"

https://chatgpt.com/ (ChatGPT)

## 0. Preperation
First, get to know the dataset and deal with missing values.
- Perform an exploratory data analysis to get to know the data set
- Preprocess the data. If there are missing values, impute them.
- Estimate the accuracy of your imputation for each feature

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('hemodat.csv')

# 1. Basic Information
print("Basic Information:")
print(df.info())
print("\nShape of the dataset:", df.shape)

In [ ]:
# 2. Summary Statistics
print("\nSummary Statistics (Numerical):")
print(df.describe())

print("\nSummary Statistics (Categorical):")
categorical_columns = df.select_dtypes(include=['object']).columns
print(df[categorical_columns].describe())

In [ ]:
# 3. Distribution Analysis
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Histograms for numerical data
df[numerical_columns].hist(bins=15, figsize=(15, 10), layout=(len(numerical_columns)//3 + 1, 3))
plt.tight_layout()
plt.show()

# Count plots for categorical data
for col in categorical_columns:
    sns.countplot(y=col, data=df)
    plt.show()

In [ ]:
# 4. Correlation Matrix
# Select only numeric columns for correlation matrix
numeric_columns = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_columns].corr()

# Visualize the correlation matrix
sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm")
plt.title("Correlation Matrix of Numeric Features")
plt.show()

In [ ]:
# 5. Feature Engineering Insights
print("\nUnique Values per Column:")
print(df.nunique())

In [ ]:
# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Calculate the number of missing values for each feature
missing_values = df.isnull().sum()

# Output the count of missing values for each feature
print("Missing values per feature:")
for feature, missing_count in missing_values.items():
    print(f"{feature}: {missing_count}")

In [ ]:
# Define the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')

# Apply KNN imputer to the DataFrame
numerical_columns = df.select_dtypes(include=['number']).columns
df[numerical_columns] = knn_imputer.fit_transform(df[numerical_columns])

# Check if all null values are imputed
foundNull = df.isnull().values.any()
if foundNull:
    raise TypeError('Found null value in DataFrame.')

# Output the cleaned DataFrame
print("DataFrame after KNN imputation:")
print(df)

In [ ]:
# Select numerical columns
numerical_columns = df.select_dtypes(include=['number']).columns
df_numerical = df[numerical_columns]

# Create a copy of the original data
original_data = df_numerical.copy()

# Introduce missing values artificially (10% missing)
np.random.seed(42)
mask = np.random.rand(*df_numerical.shape) < 0.1  # Mask for 10% missing
df_missing = df_numerical.copy()
df_missing[mask] = np.nan

# Apply KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')
df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_missing), columns=numerical_columns)

# Compare imputed values with original values
print(f"Mean Squared Error for each feature:")
for col in numerical_columns:
    # Calculate error only for artificially missing values
    mask_col = mask[:, df_numerical.columns.get_loc(col)]
    mse = mean_squared_error(original_data[col][mask_col], df_imputed[col][mask_col])
    print(f"'{col}': {mse}")

## 1. Anomaly Detection
Since medical conditions that lead to the rejection of a donor are rare (luckily) and can be very
versatile. It is near impossible to categorize every possible condition. Hence, it would be useful to have an anomaly
detection algorithm in place as a safety mechanism to detect suspicious blood samples for further testing.
- Train an anomaly detection model based only on valid blood donors without a medical condition.
- Evaluate the accuracy of your anomaly detection by testing it also on donors with a medical condition.
- Perform a PCA to visualize the true / false positive and true / false negative predictions as well as the decision
boundary of your anomaly detection. How much variance is explained by the first two main components? 

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Filter data for valid blood donors
donors = df[df['Category'] == '0=Blood Donor']

# Filter data for non-donors (anomalous category)
non_donors = df[df['Category'] != '0=Blood Donor']

# Numerical columns for modeling
numerical_columns = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

# Features for anomaly detection
X_train_valid, X_test_valid = train_test_split(donors[numerical_columns], test_size=0.2, random_state=42) # Train only on valid donors
X_test = non_donors[numerical_columns]  # Use non-donors for testing

# Define and train Isolation Forest
iso_forest = IsolationForest(random_state=42, contamination=0.1)  # Assuming 10% contamination
iso_forest.fit(X_train_valid)

# Predict anomalies on the combined dataset
X_test_valid_with_labels = donors.loc[X_test_valid.index]
X_test_with_labels = non_donors.loc[X_test.index]
combined_data = pd.concat([X_test_valid_with_labels, X_test_with_labels], axis=0)
combined_features = combined_data[numerical_columns]
predictions = iso_forest.predict(combined_features)

# Map predictions to binary format (1: normal, -1: anomaly)
pred_binary = np.where(predictions == 1, 1, 0)  # 1: Valid, 0: Anomaly
true_labels = np.where(combined_data['Category'] == '0=Blood Donor', 1, -1)
true_binary = np.where(true_labels == 1, 1, 0)

# Classification report
print("Classification Report:")
print(classification_report(true_binary, pred_binary, target_names=['Anomaly', 'Valid']))

# Calculate the counts of true/false positives and negatives
tp = np.sum((true_binary == 1) & (pred_binary == 1))  # True Positives
fp = np.sum((true_binary == 0) & (pred_binary == 1))  # False Positives
tn = np.sum((true_binary == 0) & (pred_binary == 0))  # True Negatives
fn = np.sum((true_binary == 1) & (pred_binary == 0))  # False Negatives

# Display the counts
print(f"Counts:")
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

# Perform PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(combined_features)

# Variance explained by PCA components
variance_explained = pca.explained_variance_ratio_
print(f"Variance explained by the first two components: {variance_explained}")

# Scatter plot with True/False Positives and Negatives
plt.figure(figsize=(12, 8))
plt.scatter(pca_data[(true_binary == 1) & (pred_binary == 1), 0], pca_data[(true_binary == 1) & (pred_binary == 1), 1], 
            c='green', label='True Positives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 0) & (pred_binary == 1), 0], pca_data[(true_binary == 0) & (pred_binary == 1), 1], 
            c='orange', label='False Positives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 0) & (pred_binary == 0), 0], pca_data[(true_binary == 0) & (pred_binary == 0), 1], 
            c='blue', label='True Negatives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 1) & (pred_binary == 0), 0], pca_data[(true_binary == 1) & (pred_binary == 0), 1], 
            c='red', label='False Negatives', alpha=0.6, edgecolor='k')
plt.title("PCA Visualization of Predictions")
plt.xlabel(f"Principal Component 1 ({variance_explained[0]:.2%} variance)")
plt.ylabel(f"Principal Component 2 ({variance_explained[1]:.2%} variance)")
plt.legend()
plt.grid(True)
plt.show()

# Decision boundary visualization using PCA components
xx, yy = np.meshgrid(
    np.linspace(pca_data[:, 0].min() - 1, pca_data[:, 0].max() + 1, 100),
    np.linspace(pca_data[:, 1].min() - 1, pca_data[:, 1].max() + 1, 100)
)
# Prepare the grid points for prediction
mesh_points = np.c_[xx.ravel(), yy.ravel()]
inverse_transformed = pca.inverse_transform(mesh_points)

# Ensure the mesh_points (inverse_transformed) have valid feature names
inverse_transformed_df = pd.DataFrame(inverse_transformed, columns=numerical_columns)

# Predict anomaly scores for the mesh points
mesh_scores = iso_forest.decision_function(inverse_transformed_df).reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(12, 8))
plt.contourf(xx, yy, mesh_scores, levels=50, cmap='coolwarm', alpha=0.5)
plt.colorbar(label='Anomaly Score')
plt.scatter(pca_data[(true_binary == 1) & (pred_binary == 1), 0], pca_data[(true_binary == 1) & (pred_binary == 1), 1], 
            c='green', label='True Positives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 0) & (pred_binary == 1), 0], pca_data[(true_binary == 0) & (pred_binary == 1), 1], 
            c='orange', label='False Positives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 0) & (pred_binary == 0), 0], pca_data[(true_binary == 0) & (pred_binary == 0), 1], 
            c='blue', label='True Negatives', alpha=0.6, edgecolor='k')
plt.scatter(pca_data[(true_binary == 1) & (pred_binary == 0), 0], pca_data[(true_binary == 1) & (pred_binary == 0), 1], 
            c='red', label='False Negatives', alpha=0.6, edgecolor='k')
plt.title("PCA Visualization with Decision Boundary")
plt.xlabel(f"Principal Component 1 ({variance_explained[0]:.2%} variance)")
plt.ylabel(f"Principal Component 2 ({variance_explained[1]:.2%} variance)")
plt.legend()
plt.grid(True)
plt.show()

## 2. Explainable Model
For your decision support your model should be explainable. Train a model with a focus on
explainability with an as simple as possible structure while still maintaining its predictive power.
- Train a decision tree classifier on the imputed data. Evaluate your model’s accuracy and visualize the tree structure to
help the hospital personal understand the decision process. Each inference should not only put out the class, but also
the decision path taken. Make the tree as simple and understandable as possible.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from scipy.sparse import find

# Prepare features and target
# Include Age, Sex, and numerical columns
numerical_columns = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']
features = ['Age', 'Sex'] + numerical_columns

# Encode 'Sex'
if df['Sex'].dtype == 'object':
    df['Sex'] = df['Sex'].map({'m': 0, 'f': 1})  # Example encoding: 0 for male, 1 for female

# Target: 'Category' (1 for valid, 0 for invalid)
df['Target'] = (df['Category'] == '0=Blood Donor').astype(int)

# Split data into training and testing sets
X = df[features]
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(max_depth=4, random_state=42)  # Adjust max_depth for simplicity
dt_classifier.fit(X_train, y_train)

# Evaluate model
y_pred = dt_classifier.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Invalid', 'Valid']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Invalid', 'Valid']).plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Visualize Decision Tree
plt.figure(figsize=(16, 10))
plot_tree(
    dt_classifier,
    feature_names=features,
    class_names=['Invalid', 'Valid'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Decision Tree Visualization")
plt.show()

# Extract decision paths for explainability
print("Decision Tree Rules:")
tree_rules = export_text(dt_classifier, feature_names=features)
print(tree_rules)

# Example of explaining individual predictions
print("\nExample Predictions and Decision Paths:")
for i in range(5):  # Show decision paths for the first 5 samples in the test set
    sample = X_test.iloc[[i]]  # Keep it as a DataFrame with proper column names
    path = dt_classifier.decision_path(sample)  # Use the correct input format
    print(f"\nSample {i+1}: {sample}")
    
    # Decode decision path
    node_indices = find(path)[1]  # Extract nodes from the sparse matrix
    print("Decision Path (Node Indices):", node_indices)



## 3. High Performance Model
This time the focus is on predictive power. Try and train a more accurate model. Is it worth
the effort?
- Train and optimize an XGBoost classifier on the imputed data.
- Use SHAP local explanation techniques on 5 selected data points and discuss the results
- Use SHAP global explanation techniques to visualize and discuss the influence of different features.
- Evaluate the XGBoost’s accuracy and compare it to the Decision Tree

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Features and target
X = df[features]  
y = df['Target']  

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost Classifier
xgb = XGBClassifier(eval_metric='logloss', random_state=42)

# Hyperparameter optimization (example grid)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(xgb, param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_xgb = grid_search.best_estimator_

# Evaluate the model
y_pred = best_xgb.predict(X_test)
print("Classification Report (XGBoost):")
print(classification_report(y_test, y_pred, target_names=['Invalid', 'Valid']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Invalid', 'Valid']).plot(cmap='Blues')
plt.title("XGBoost Confusion Matrix")
plt.show()

# Output the best parameters
print("Best Parameters:", grid_search.best_params_)

In [ ]:
import shap

# Initialize SHAP explainer
explainer = shap.Explainer(best_xgb)

# Select 5 random test samples for explanation
sample_indices = X_test.sample(5, random_state=42).index
sample_data = X_test.loc[sample_indices]

# Explain predictions
shap_values = explainer(sample_data)

# Visualize SHAP explanations for each selected data point
for i, index in enumerate(sample_indices):
    print(f"SHAP explanation for test sample {i + 1}:")
    shap.waterfall_plot(shap_values[i])

In [ ]:
# Compute SHAP values for the entire test set
shap_values_global = explainer(X_test)  

# Global summary plot
shap.summary_plot(shap_values_global, X_test)

# Bar plot for mean absolute SHAP values
shap.summary_plot(shap_values_global, X_test, plot_type="bar")

## 4. Combined Model
Put all components into a single model artifact for deployment such that clinic personal has all important
information at hand to make an informed decision.
- Combine the XGBoost, Decision Tree and Anomaly Detection in a single model class including all necessary methods (fit,
predict…). The Decision Tree provides an explainable assistance for the hospital personal and the XGBoost (probably) a more
accurate classification. The Anomaly Detection increases the robustness of the model for conditions that have not been
explicitly trained or for human errors. Generate a few test anomalies to check your detection.
- Evaluate, discuss and plot the performance of your combined model.

In [ ]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

class CombinedModel:
    def __init__(self):
        # Initialize models
        self.xgb = XGBClassifier(eval_metric='logloss', random_state=42)
        self.decision_tree = DecisionTreeClassifier(random_state=42)
        self.anomaly_detector = IsolationForest(random_state=42)
    
    def fit(self, X_train, y_train):
        """
        Fit the XGBoost, Decision Tree, and Anomaly Detection models.
        """
        print("Fitting XGBoost...")
        self.xgb.fit(X_train, y_train)
        
        print("Fitting Decision Tree...")
        self.decision_tree.fit(X_train, y_train)
        
        print("Fitting Anomaly Detector...")
        self.anomaly_detector.fit(X_train)  # Fit Isolation Forest only on features
    
    def predict(self, X):
        """
        Predict using the combined model.
        """
        # Step 1: Anomaly Detection
        anomalies = self.anomaly_detector.predict(X)  # -1 indicates anomaly, 1 indicates normal
        
        # Step 2: Decision Tree Prediction (for interpretation)
        dt_preds = self.decision_tree.predict(X)
        
        # Step 3: XGBoost Prediction (for accuracy)
        xgb_preds = self.xgb.predict(X)
        
        # Step 4: Combine results
        combined_preds = []
        for i, anomaly in enumerate(anomalies):
            if anomaly == -1:
                combined_preds.append("Anomaly")
            else:
                combined_preds.append(xgb_preds[i])
        
        return combined_preds, dt_preds  # Return XGBoost predictions and Decision Tree predictions separately
    
    def evaluate(self, X_test, y_test):
        """
        Evaluate the performance of the combined model.
        """
        combined_preds, dt_preds = self.predict(X_test)
        
        # Exclude anomalies for evaluation
        normal_indices = [i for i, pred in enumerate(combined_preds) if pred != "Anomaly"]
        filtered_preds = [combined_preds[i] for i in normal_indices]
        filtered_y_test = y_test.iloc[normal_indices]
        
        print("Evaluation for Normal Predictions:")
        print(classification_report(filtered_y_test, filtered_preds))
        
        print("Confusion Matrix for Normal Predictions:")
        print(confusion_matrix(filtered_y_test, filtered_preds))
    
    def detect_anomalies(self, X):
        """
        Detect anomalies in the data.
        """
        anomaly_predictions = self.anomaly_detector.predict(X)
        anomalies = np.where(anomaly_predictions == -1)[0]  # Index of anomalies
        print(f"Detected {len(anomalies)} anomalies.")
        return anomalies


In [ ]:
model = CombinedModel()
model.fit(X_train, y_train)

In [ ]:
predictions, dt_interpretations = model.predict(X_test)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
anomalies = model.detect_anomalies(X_test)

In [ ]:
# Create anomalies by adding random noise
def generate_anomalies(X, n_samples=10, noise_level=10):
    np.random.seed(42)
    anomalies = X.sample(n=n_samples).copy()
    for col in X.columns:
        anomalies[col] += np.random.uniform(-noise_level, noise_level, size=n_samples)
    return anomalies

# Add anomalies to test the detection
X_test_anomalies = generate_anomalies(X_test, n_samples=10)
anomalies_detected = model.detect_anomalies(X_test_anomalies)